In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from scipy import signal
from scipy.optimize import minimize

import random

import h5py
from IPython.utils import io

from joblib import Parallel, delayed
from tqdm import tqdm
import sys

In [2]:
%matplotlib widget

%reload_ext autoreload
%autoreload 2

In [3]:
sys.path.append('../../../../nadavp/new_trap/Tools/')
sys.path.append('../../../../nadavp/new_trap/Tools/StatFramework/')
sys.path.append('../../../../nadavp/opt_lev_analysis/lib/')
import BeadDataFile
from discharge_tools import *

In [4]:
from likelihood_calculator import likelihood_analyser
from likelihood_calculator import GravityFramework
from likelihood_calculator import auxiliary_functions as aux
gfw = GravityFramework.GravityFramework()

In [8]:
import bead_util as bu
import transfer_func_util as tf
import configuration as config
from discharge_tools import correlation

### Parameters

In [18]:
## some are coming from the notebook: Old trap - 20211001 - All calibration with comments

scaleZ = 128

voltage_calib = 25
voltage_200 = 200

E1, E2 = (131, 195)
E_tot = E1+E2
factor = 0.621

In [19]:
etha = 1/scaleZ*E_tot*voltage_calib*1.6e-19 ## convert bits to force
to_epsilon = 1/(2*131*1.6e-19*voltage_200) ## this is 2E1 - the convertion factor from A to epsilon

### Look at neutrality data

In [93]:
def get_amp_wraperZ(i, fname):
    try:
        return get_ampZ(i, fname)
    except:
        return 0

In [119]:
# set the drive and settings for the correlation
bb = bu.DataFile();

def get_ampZ(i, foldername):
    
    fname = foldername+'Z_200Vpp_'+str(i)+'.h5'
    bb.load(fname);
    bb.load_other_data()
    
    bandwidth = 2
    drive_freq = 139
    fsamp = 5000
    sigamp = 1
    
    ## extract the drive phase - depends on which electrode is driving
    jj = i%2
    freq2 = np.fft.rfftfreq(len(bb.other_data[1+jj]), d=1./5000)
    fft_angles = np.angle(np.fft.rfft(bb.other_data[1+jj]))
    phi_tmp2 = (fft_angles[freq2==drive_freq]+np.pi/2)%(2*np.pi)+4.8
    
    nsamp = len( bb.pos_data[2])
    tarr = np.arange(0, nsamp/fsamp, 1.0/fsamp)
    sig = sigamp * np.sin(2.0 * np.pi * drive_freq * tarr+phi_tmp2)

    return correlation(sig, bb.pos_data[2] , fsamp=5000, fdrive=139, filt=True, band_width=bandwidth)[0]

In [95]:
base_folder = r'/data/old_trap/20211001/bead1/'
fname = base_folder+r'neutrality6/Z_150Vpp_alternate_139Hz_spin_XY_93kHz_200Vpp_delay_3sec_with_height_fb/'
get_ampZ(0, fname)

-9.8534873435342192

In [120]:
base_folder = r'/data/old_trap/20211001/bead1/'
fname = base_folder+r'neutrality8/Z_200Vpp_alternate_139Hz_spin_XY_117kHz_175Vpp_delay_7sec_with_height_fb_redo3/'
get_ampZ(0, fname)

12.277606027269531

### Loading all datasets taken with ~100kHz spinning field 

In [96]:
## the prefix of the some of the folders is Z_150Vpp, but that is raelly 200V data
## I messed up the the folder names, it is spinning at 175Vpp with 200Vpp oscillating field 
## (it has been validated using the digitized electrode data and also by the filenames which are 200Vpp)
## spinning is either 93kHz or 103kHz
base_folder = r'/data/old_trap/20211001/bead1/'
fnames = [base_folder+r'neutrality6/Z_150Vpp_alternate_139Hz_spin_XY_93kHz_200Vpp_delay_3sec_with_height_fb/',
base_folder+r'neutrality6/Z_150Vpp_alternate_139Hz_spin_XY_103kHz_200Vpp_delay_3sec_with_height_fb/',
base_folder+r'neutrality6/Z_150Vpp_alternate_139Hz_spin_XY_103kHz_200Vpp_delay_3sec_with_height_fb_redo/',
# base_folder+r'neutrality6/Z_150Vpp_alternate_139Hz_spin_XY_103kHz_200Vpp_delay_3sec_with_height_fb_redo2/', ## rga incident
base_folder+r'neutrality6/Z_200Vpp_alternate_139Hz_spin_XY_103kHz_175Vpp_delay_3sec_with_height_fb_redo3/',
base_folder+r'neutrality6/Z_200Vpp_alternate_139Hz_spin_XY_103kHz_175Vpp_delay_3sec_with_height_fb_redo4/']

files = [2000, 2000, 2000, 1000, 1000]

In [97]:
ams_neutrality6 = []
for i_folder in range(len(files)):
    amps_tmp = np.array(Parallel(n_jobs=40)(delayed(get_amp_wraperZ)(i, fnames[i_folder]) for i in tqdm(range((files[i_folder])))))
    ams_neutrality6 += list(np.array(amps_tmp)*etha)
ams_neutrality6 = np.array(ams_neutrality6)

 38%|███▊      | 760/2000 [00:15<00:29, 41.50it/s]/home/analysis_user/.local/lib/python3.6/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
100%|██████████| 1000/1000 [00:15<00:00, 44.17it/s]


In [98]:
## getting rid of one corrupted file - have to extract two files in order to keep the odd always plus electrode
ams_neutrality6_2 = list(ams_neutrality6)
ams_neutrality6_2.pop(692)
ams_neutrality6_2.pop(692)
ams_neutrality6 = np.array(ams_neutrality6_2)

In [121]:
## higher spinning - 117kHz
base_folder = r'/data/old_trap/20211001/bead1/'
fnames = [base_folder+r'neutrality8/Z_200Vpp_alternate_139Hz_spin_XY_117kHz_175Vpp_delay_5sec_with_height_fb/',
base_folder+r'neutrality8/Z_200Vpp_alternate_139Hz_spin_XY_117kHz_175Vpp_delay_7sec_with_height_fb_redo2/',
base_folder+r'neutrality8/Z_200Vpp_alternate_139Hz_spin_XY_117kHz_175Vpp_delay_7sec_with_height_fb_redo3/',
base_folder+r'neutrality8/Z_200Vpp_alternate_139Hz_spin_XY_117kHz_175Vpp_delay_7sec_with_height_fb_redo4/',
base_folder+r'neutrality8/Z_200Vpp_alternate_139Hz_spin_XY_117kHz_175Vpp_delay_7sec_with_height_fb_redo5/']

files = [2000, 2000, 2000, 2000, 438]

In [122]:
ams_neutrality8 = []
for i_folder in range(len(files)):
    amps_tmp = np.array(Parallel(n_jobs=40)(delayed(get_amp_wraperZ)(i, fnames[i_folder]) for i in tqdm(range((files[i_folder])))))
    ams_neutrality8 += list(np.array(amps_tmp)*etha)
ams_neutrality8 = np.array(ams_neutrality8)

 24%|██▍       | 480/2000 [00:03<00:11, 135.37it/s]/home/analysis_user/.local/lib/python3.6/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
100%|██████████| 438/438 [00:03<00:00, 132.52it/s]


In [101]:
## higher spinning - 117kHz
## after improving the switching on/off of the electrodes to allow more off time per configuration
base_folder = r'/data/old_trap/20211001/bead1/'
fnames = [base_folder+r'neutrality9/Z_200Vpp_alternate_139Hz_spin_XY_117kHz_175Vpp_delay_4sec_with_height_fb/',
base_folder+r'neutrality9/Z_200Vpp_alternate_139Hz_spin_XY_117kHz_175Vpp_delay_4sec_with_height_fb_redo/',
base_folder+r'neutrality9/Z_200Vpp_alternate_139Hz_spin_XY_117kHz_175Vpp_delay_4sec_with_height_fb_redo2/',
base_folder+r'neutrality9/Z_200Vpp_alternate_139Hz_spin_XY_117kHz_175Vpp_delay_4sec_with_height_fb_redo3/',
base_folder+r'neutrality9/Z_200Vpp_alternate_139Hz_spin_XY_117kHz_175Vpp_delay_6sec_with_height_fb_redo4/',
base_folder+r'neutrality9/Z_200Vpp_alternate_139Hz_spin_XY_117kHz_175Vpp_delay_4sec_with_height_fb_redo5/',
base_folder+r'neutrality9/Z_200Vpp_alternate_139Hz_spin_XY_117kHz_175Vpp_delay_6sec_with_height_fb_redo6/']

files = [80, 2000, 2000, 2000, 2000, 4000, 2200]

In [102]:
ams_neutrality9 = []
for i_folder in range(len(files)):
    amps_tmp = np.array(Parallel(n_jobs=40)(delayed(get_amp_wraperZ)(i, fnames[i_folder]) for i in tqdm(range((files[i_folder])))))
    ams_neutrality9 += list(np.array(amps_tmp)*etha)
ams_neutrality9 = np.array(ams_neutrality9)

100%|██████████| 2200/2200 [00:36<00:00, 62.77it/s]


In [103]:
# different spin voltage -- skipped that one accidently before it adds a bit of sensitity
base_folder = r'/data/old_trap/20211001/bead1/'
fnames = [base_folder+r'neutrality11/Z_200Vpp_alternate_139Hz_spin_XY_117kHz_200Vpp_delay_6sec_with_height_fb/']

files = [2500]

In [104]:
ams_neutrality11 = []
for i_folder in range(len(files)):
    amps_tmp = np.array(Parallel(n_jobs=40)(delayed(get_amp_wraperZ)(i, fnames[i_folder]) for i in tqdm(range((files[i_folder])))))
    ams_neutrality11 += list(np.array(amps_tmp)*etha)
ams_neutrality11 = np.array(ams_neutrality11)

100%|██████████| 2500/2500 [00:44<00:00, 56.66it/s]


In [109]:
ams_neutrality9

array([ -4.13217680e-18,   2.64993730e-17,  -3.43001315e-17, ...,
         1.36582490e-16,   1.70878657e-16,   1.34305398e-16])

In [134]:
amps_all = np.array(list(ams_neutrality6)+list(ams_neutrality8)+list(ams_neutrality9)+list(ams_neutrality11))

### Data Analysis

In [135]:
data_plus = amps_all[::2]
data_minus = amps_all[1::2]
data_A = data_plus-factor*data_minus

_, ax= plt.subplots(figsize=(9.5,4))
ax.scatter(np.arange(len(data_plus))*2*10/3600, data_plus, label='200Vpp-z-plus')
ax.scatter(np.arange(len(data_minus))*2*10/3600, data_minus, label='200Vpp-z-minus', alpha=0.6)
ax.plot(np.arange(len(data_A))*2*10/3600, np.arange(len(data_A))*0, '--k')
ax.set(xlabel='Integration time [hr.]', ylabel=r'$\epsilon$')
ax.legend()
print('1st harmonic, electrode 1: mean, std: ', np.mean(data_plus), np.std(data_plus)/np.sqrt(len(data_plus)))
print('1st harmonic, electrode 2: mean, std: ', np.mean(data_minus), np.std(data_minus)/np.sqrt(len(data_minus)))
print('1st harmonic, A parameter: mean, std: ', np.mean(data_A), np.std(data_A)/np.sqrt(len(data_A)))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

1st harmonic, electrode 1: mean, std:  -2.43276659404e-17 6.67689389436e-19
1st harmonic, electrode 2: mean, std:  -3.96299408643e-17 8.67919148229e-19
1st harmonic, A parameter: mean, std:  2.82527336344e-19 4.25223903703e-19


In [136]:
N = len(data_plus)
data_shifted = np.array([(data_plus[i]+data_plus[i-1])/2 for i in range(1,N)])
data_A_shifted = data_shifted-factor*data_minus[:-1]

_, ax= plt.subplots(figsize=(9.5,4))
ax.scatter(np.arange(len(data_A))*2*10/3600, data_A, label='200Vpp-z-plus, A parameter')
ax.scatter(np.arange(len(data_A_shifted))*2*10/3600, data_A_shifted, label='200Vpp-z, corrected A parameter', alpha=0.6)

ax.plot(np.arange(len(data_A))*2*10/3600, np.arange(len(data_A))*0, '--k')
ax.set(xlabel='Integration time [hr.]', ylabel=r'$\epsilon$')
ax.legend()
print('1st harmonic, A parameter: mean, std: ', np.mean(data_A), np.std(data_A)/np.sqrt(len(data_A)))
print('1st harmonic, corrected A: mean, std: ', np.mean(data_A_shifted), np.std(data_A_shifted)/np.sqrt(len(data_A_shifted)))
print('1st harmonic, A parameter[epsilon]: mean, std: ', np.mean(data_A_shifted)*to_epsilon, to_epsilon*np.std(data_A_shifted)/np.sqrt(len(data_A_shifted)))


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

1st harmonic, A parameter: mean, std:  2.82527336344e-19 4.25223903703e-19
1st harmonic, corrected A: mean, std:  2.84227094091e-19 3.40528316042e-19
1st harmonic, A parameter[epsilon]: mean, std:  3.39011324059e-05 4.06164499095e-05


### Fitting

In [137]:
from scipy.optimize import curve_fit
from scipy import asarray as ar,exp

n_bins = 29
range_pram = 0.02
_,ax = plt.subplots()
entries, bins, patches = ax.hist(data_A_shifted*to_epsilon, bins=n_bins, range=(-range_pram, range_pram), label='200Vpp, all frequencies')
ax.set(xlabel=r'Force [N]')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[Text(0.5, 0, 'Force [N]')]

In [138]:
## gaussian fit to the histogram

bins_fit =[(bins[i+1]-bins[i])/2+bins[i] for i in range(len(entries))]

titles = ['', '', '']

for i,ee in enumerate([entries]):
    x = bins_fit
    y= np.array(ee)
    sigma_bin = np.sqrt(ee)
    
    n = len(x)                          #the number of data
    mean = 0                  #note this correction
    sigma = 0.03       #note this correction

    def gaus(x,a,x0,sigma):
        return a*np.exp(-(x-x0)**2/(2*sigma**2))

    popt,pcov = curve_fit(gaus,x,y,p0=[1,mean,sigma])
    ax.plot(x,gaus(x,*popt),'ro:',label='fit')

    print("mean: ", popt[1], " ,std (on the mean): ", popt[2]/np.sqrt(len(data_A_shifted)), " ,sigma: ", popt[2])
    print("std from the covariance matrix: ", np.sqrt(pcov[1,1]))
    
    nucleons = 420e-12*6e23/2
    print('\nNeutrality: ', 1/nucleons*popt[2]/np.sqrt(len(data_A_shifted)))
    print('Factor needed for neutrality best limit: ', 1/nucleons*popt[2]/np.sqrt(len(data_A_shifted))/1e-21)

mean:  3.58133536701e-05  ,std (on the mean):  4.02392834684e-05  ,sigma:  0.00518556200472
std from the covariance matrix:  3.69451285839e-05

Neutrality:  3.19359392606e-19
Factor needed for neutrality best limit:  319.359392606
